In [1]:
from django.shortcuts import render

# Create your views here.
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_exempt
#from django.db.transaction import commit_on_success
from django.db import transaction

'''
from .models import Embedding
from .models import Translation
from .models import Valid
'''
import json
import os

#main_path = "/Volumes/TOSHIBA EXT/temp/kbqa_portable_prj"
#main_path = "/User/kbqa_portable_prj"
main_path = "/temp/kbqa_portable_prj"

import json
import os
from functools import lru_cache, partial, reduce

import numpy as np
import pandas as pd
import sqlite_utils
from rdflib.graph import Graph
from rdflib_hdt import HDTStore
from timer import timer
from tqdm import tqdm

os.environ["DP_SKIP_NLTK_DOWNLOAD"] = "True"

import inspect
import json
import logging
import os
import re
import sys
from collections import defaultdict
from functools import reduce
from itertools import permutations, product

import numpy as np
import pandas as pd
from deeppavlov import build_model, configs
from deeppavlov.core.commands.infer import *
from deeppavlov.core.commands.utils import *
from deeppavlov.core.common.file import *
from deeppavlov.models.kbqa.wiki_parser import *
from rapidfuzz import fuzz
from scipy.special import softmax


logging.disable(sys.maxsize)

import csv
import gzip
import inspect
import logging
import math
import os
import re
import shutil
import sys
from collections import Counter, defaultdict, namedtuple
from copy import deepcopy
from datetime import datetime
from functools import partial, reduce

import editdistance
import networkx as nx
import numpy as np
import pandas as pd
import sqlite_utils
import synonyms
import torch.nn as nn
from deeppavlov import build_model, configs
from deeppavlov.core.commands.infer import *
from deeppavlov.core.common.file import *
from deeppavlov.models.kbqa.query_generator import *
from deeppavlov.models.kbqa.query_generator_base import *
from deeppavlov.models.kbqa.wiki_parser import *
from pandas.io.common import _stringify_path
from scipy.special import softmax
from sentence_transformers import InputExample, LoggingHandler, util
from sentence_transformers.util import pytorch_cos_sim
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import \
    CECorrelationEvaluator
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers import (
    AdapterConfig,
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    HfArgumentParser,
    MultiLingAdapterArguments,
    PreTrainedTokenizerFast,
    Trainer,
    TrainingArguments,
    set_seed,
)

from dataclasses import dataclass, field

import jieba
from hashlib import sha512

pd.set_option('max_colwidth', 60)
pd.set_option("max_columns", 20)

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    task_name: Optional[str] = field(default="ner", metadata={"help": "The name of the task (ner, pos...)."})
    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a csv or JSON file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate on (a csv or JSON file)."},
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to predict on (a csv or JSON file)."},
    )
    text_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of text to input in the file (a csv or JSON file)."}
    )
    label_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of label to input in the file (a csv or JSON file)."}
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    label_all_tokens: bool = field(
        default=False,
        metadata={
            "help": "Whether to put the label for one word on all tokens of generated by that word or just on the "
            "one (in which case the other tokens will have a padding index)."
        },
    )
    return_entity_level_metrics: bool = field(
        default=False,
        metadata={"help": "Whether to return all the entity levels during evaluation or just the overall ones."},
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        self.task_name = self.task_name.lower()

import sys
sys.path.insert(0, main_path)

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/opt/conda/envs/kbqa_env/lib/python3.7/site-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/opt/conda/envs/kbqa_env/lib/python3.7/site-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/opt/conda/envs/kbqa_env/lib/python3.7/site-packages/synonyms/data/words.vector.gz] ...


/opt/conda/envs/kbqa_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
p2 = os.path.join("/temp", "kbqa-explore")
assert os.path.exists(p2)
sys.path.insert(0, p2)

#p3 = os.path.join(main_path, "kbqa-explore/linker_entities.pkl")
p3 = os.path.join("/temp", "kbqa-explore/linker_entities.pkl")
assert os.path.exists(p3)
#zh_linker_entities = load_pickle("kbqa-explore/linker_entities.pkl")
zh_linker_entities = load_pickle(p3)
zh_linker_entities.num_entities_to_return = 5

In [4]:
zh_linker_entities(
    [["苹果"]]
)

order_dict_for_merge :
{}
srtd_with_ratios :
[(309705, 'statement', 0, 1, 100.0, 37461), (5978728, 'Q312', 0, 1, 100.0, 8), (867427, 'Q5226961', 0, 1, 100.0, 6), (6063280, 'Q27500714', 0, 1, 100.0, 4), (611832, 'Q421253', 0, 1, 80.0, 8), (5936477, 'Q1054357', 0, 1, 80.0, 6), (4937440, 'Q1612679', 0, 1, 80.0, 5), (4535924, 'Q104819', 0, 1, 80.0, 5), (3610357, 'Q65571', 0, 1, 80.0, 4), (1864019, 'Q1144556', 0, 1, 80.0, 4), (1586669, 'Q483549', 0, 1, 80.0, 4), (769615, 'Q4781156', 0, 1, 80.0, 4), (2827822, 'Q15227818', 0, 1, 80.0, 3), (2732690, 'Q1358074', 0, 1, 80.0, 3), (1175213, 'Q305918', 0, 1, 66.66666666666667, 15), (2202251, 'Q10915001', 0, 1, 66.66666666666667, 7), (924325, 'Q504893', 0, 1, 66.66666666666667, 6), (4485018, 'Q18010946', 0, 1, 66.66666666666667, 5), (321355, 'Q213710', 0, 1, 66.66666666666667, 4), (4741273, 'Q148737', 0, 1, 66.66666666666667, 4), (1745143, 'Q270285', 0, 1, 66.66666666666667, 4), (4417578, 'Q1407528', 0, 1, 66.66666666666667, 3), (5817352, 'Q2613423'

([[['statement', 'Q312', 'Q5226961', 'Q27500714', 'Q421253']]],
 [[[0.01, 0.01, 0.01, 0.01, 0.01]]])

In [5]:
zh_linker_entities(
    [["李存勖"]]
)

order_dict_for_merge :
{}
srtd_with_ratios :
[(3045628, 'Q706815', 0, 1, 100.0, 8)]
----------------------------------------------------------------------------------------------------


([[['Q706815']]], [[[0.01]]])

In [12]:
output = zh_linker_entities(
    [["李芳果"]]
)

order_dict_for_merge :
{}
srtd_with_ratios :
[(3796814, 'Q485556', 0, 1, 100.0, 5)]
----------------------------------------------------------------------------------------------------


In [15]:
np.asarray(output[0]).reshape([-1]).tolist()

['Q485556']

In [16]:
output = zh_linker_entities(
    [[""]]
)

In [17]:
np.asarray(output[0]).reshape([-1]).tolist()

['None']

In [18]:
import requests
import json

In [21]:
qst = "哈利波特"
rep = requests.post(
    url = "http://localhost:8800/zh_entity_link",
    data = {
        "entity_str":  qst
    }
)
json.loads(rep.content.decode())

[['Q216930', ['哈利·波特', '哈利波特 (系列電影)', '哈利波特 (電影系列)', '哈利波特', '哈利波特系列電影']],
 ['Q3244512', ['哈利·波特', '哈利波特']],
 ['Q8337', ['哈利·波特', '哈利波特']],
 ['Q46751',
  ['哈利·波特与火焰杯',
   '三强争霸赛',
   '哈利·波特—火盃的考驗',
   '哈利波特-火盃的考驗',
   '哈利波特4',
   '哈利波特—火盃的考驗',
   '哈利波特与火焰杯',
   '哈利波特－火盃的考驗',
   '哈利波特：火盃的考驗',
   '火盃的考驗']],
 ['Q80817',
  ['哈利·波特与凤凰社',
   'Harry Potter and the Order of the Phoenix',
   '哈利·波特—鳳凰會的密令',
   '哈利波特-鳳凰會的密令',
   '哈利波特5',
   '哈利波特—鳳凰會的密令',
   '哈利波特与凤凰社',
   '哈利波特鳳凰會的密令',
   '哈利波特－鳳凰會的密令',
   '鳳凰會的密令']]]

In [22]:
qst = "哈利波特"
rep = requests.post(
    url = "http://localhost:8800/zh_entity_link",
    data = {
        "entity_str":  ""
    }
)
json.loads(rep.content.decode())

[]

In [27]:
req = requests.post(
    url = "http://localhost:9999/search_qa",
    data = {
        "answer": "阿尔斯兰",
        "question": "谁是大塞尔柱苏丹?",
        "k": "50"
    }
)
req = json.loads(req.content.decode())
req

{'output': [{'entity': '侯格尔奥格·路易',
   'question': '谁是大不列颠王国的国王?',
   'title': '不伦瑞克-吕讷堡选侯国',
   'context': ' 1705年，选帝侯格尔奥格·路易继承了吕讷堡公国与萨克森-劳恩堡（Sachsen-Lauenburg）公国，这是继承自他的叔叔不伦瑞克-吕讷堡的格尔奥格·威廉的  1715年，他又从丹麦国王弗雷德里克四世手里购买了不莱梅-费尔登（Bremen-Verden）公国（通过1719年大北方战争后签订的斯德哥尔摩条约），由此他昔日的内陆选侯国得以扩展到北海  在1714，格奥尔格·路易成为大不列颠王国的国王，称乔治一世，使得选侯国和大不列颠王国被置于一个共主邦联中 与此同时，选帝侯本身的领土也扩张了，因为选侯国于1719年依据法理购买了原属于瑞典的不莱梅公国和瓦尔登公国  乔治一世死于1727年，他的儿子乔治二世继承了选帝侯与大不列颠王国 1728年，神圣罗马帝国皇帝查理六世正式承认乔治二世的选帝侯地位（即承认了他对土地的所有权以换取效忠），同时乔治二世恢复了对萨克森-劳恩堡的主权，这是由汉诺威选侯国于1689年从法理上继承自吕讷堡公国的',
   'score': 0.9922945499420166},
  {'entity': '阿卜杜勒·哈米德',
   'question': '谁是奥斯曼帝国的苏丹?',
   'title': '阿布杜勒哈米德一世',
   'context': ' 阿卜杜勒哈米德一世（奥斯曼土耳其语：عبد الحميد اول，土耳其语：I. Abdülhamid，1725年3月20日－1789年4月7日）是奥斯曼帝国的苏丹，1774年至1789年在位  阿卜杜勒·哈米德一世为苏丹艾哈迈德三世的儿子，在其生命的头43年，他一直按照帝国的传统被其兄长和前任穆斯塔法三世监禁在后宫中（这一制度始于苏丹塞利姆二世） 当穆斯塔法三世去世后，阿卜杜勒·哈米德被官员们从监禁地拉出来，继承了苏丹和哈里发的称号 上任首先签订库楚克开纳吉条约，结束与俄罗斯的战争 此条约文本混乱，原始版本俄罗斯文、奥斯曼文、意大利文并不一致，直到80年后，此条约才有一致的英文翻译 1783年，俄罗斯凯瑟琳大帝的情夫格里高利·波将金根据此条约兼并克里米亚  阿卜杜勒

In [28]:
req_df = pd.DataFrame(req["output"])
req_df

,entity,question,title,context,score
0,侯格尔奥格·路易,谁是大不列颠王国的国王?,不伦瑞克-吕讷堡选侯国,1705年，选帝侯格尔奥格·路易继承了吕讷堡公国与萨克森-劳恩堡（Sachsen-Lauenburg）公国，这...,0.992295
1,阿卜杜勒·哈米德,谁是奥斯曼帝国的苏丹?,阿布杜勒哈米德一世,阿卜杜勒哈米德一世（奥斯曼土耳其语：عبد الحميد اول，土耳其语：I. Abdülhamid，172...,0.992153
2,斯捷潘诺斯·奥尔别,谁说塞尔柱突厥人焚毁了巴加别尔德上万本亚美尼亚手抄本?,梅斯罗普·马什托茨写本研究所,它是世界上拥有最丰富的中世纪手抄本和书籍的储存库之一，其收藏神学、哲学、史学、医学、文学和宇宙学等多个领域的亚...,0.991654
3,沙一世,谁是下任苏丹继承人?,阿尔普·阿尔斯兰,1063年，苏丹图赫里勒去世，贵族们决定让阿尔斯兰的幼弟苏莱曼·恰格勒继任苏丹 对此不满的阿尔斯兰，与同样觊觎...,0.984105
4,薛西斯,谁是大流士的儿子?,贝尔塞波利斯,[7]根据吉恩·R·加思韦特（Gene R. Garthwaite）的说法，苏萨宫是大流士建设波斯波利斯时用来模...,0.982381
5,阿尔斯兰,谁是大塞尔柱苏丹?,阿尔普·阿尔斯兰,1063年，苏丹图赫里勒去世，贵族们决定让阿尔斯兰的幼弟苏莱曼·恰格勒继任苏丹 对此不满的阿尔斯兰，与同样觊觎...,0.965975
6,李维拉,谁是大联盟的终结投手?,快速球,这种球路介于快速直球与滑球之间，通常是在投球时要投得比滑球还快，而且比典型的快速直球要多做些动作 一般投卡特球的...,0.042167
7,穆拉德,谁是苏丹?,奥斯曼帝国的崛起,约翰六世在1347年至1354年间与约翰五世共治，奥尔汗进攻盖利博卢半岛，使奥斯曼取得在欧洲的第一个据点[13]...,0.033937
8,小汉斯·霍尔拜因,谁是大主教肖像的艺术家?,兰贝斯宫,约翰·莫顿在1495年建筑了一个都铎式的砖块大门 雷吉纳尔德·博勒在1558年死后在这里放置了40天 庭院里存有...,0.018204
9,阿诺德·贝格施特雷瑟,谁是大学的社会文化研究学院?,弗赖堡大学,经过战时关闭，二战结束后大学重开 遭受战争重创的大学在艰难的条件下进行了重建 为自然科学所新建造的楼宇也在“学...,0.015583


In [29]:
req_df["entity"].value_counts()

陈慧娴             2
基督              2
张思敬             1
戈登·麦克伦南         1
阿诺斯             1
南斯拉夫            1
白崇禧             1
李维拉             1
胡国兴             1
陈名夏             1
斯捷潘诺斯·奥尔别       1
穆拉德             1
西格提·约瑟夫         1
乌特纳             1
乔治亚·欧姬芙         1
吉尔吉斯玛纳普         1
小汉斯·霍尔拜因        1
约瑟夫·休姆          1
阿诺德·贝格施特雷瑟      1
艾莉珊德            1
肯尼·基为           1
朱德              1
曼努埃尔·戈麦斯·莫林（    1
1170            1
阿尔斯兰            1
金               1
许仕仁             1
薛西斯             1
阿卜杜勒·哈米德        1
罗果仁             1
1064年4月27日      1
沙一世             1
埃塞克·霍普金斯        1
蒙古              1
张亚勤             1
伊努·L·强森         1
侯格尔奥格·路易        1
安东尼·范·戴克        1
乌尔里希·冯·均林根      1
张献忠             1
翁玉林             1
皮埃尔·吉罗德         1
Jorge           1
塞奥门·朱达          1
Name: entity, dtype: int64

In [31]:
qst = "哈利波特"
qst = "薛西斯"
rep = requests.post(
    url = "http://localhost:8800/zh_entity_link",
    data = {
        "entity_str":  qst
    }
)
json.loads(rep.content.decode())

[['Q1238572', ['薛西斯']],
 ['Q744212', ['薛西斯']],
 ['Q129165',
  ['薛西斯一世', '亚哈沙鲁', '克塞耳克塞斯', '泽克西斯一世', '泽克西斯大帝', '澤克西斯一世', '薛西斯大帝']],
 ['Q272043', ['薛西斯二世', '泽克西斯二世']],
 ['Q296434', ['貝蘇斯', '贝索斯', '贝苏斯', '阿塔薛西斯四世', '阿尔塔薛西斯四世']]]

In [32]:
qst = "哈利波特"
qst = "薛西斯"
qst = "阿尔斯兰"
rep = requests.post(
    url = "http://localhost:8800/zh_entity_link",
    data = {
        "entity_str":  qst
    }
)
json.loads(rep.content.decode())

[['Q28418938', ['阿尔斯兰']],
 ['Q16904388', ['阿赫马德·阿尔斯兰汗']],
 ['Q25530712', ['阿尔斯兰·伊列克·纳斯尔']]]

In [33]:
qst = "斯捷潘诺斯·奥尔别"
rep = requests.post(
    url = "http://localhost:8800/zh_entity_link",
    data = {
        "entity_str":  qst
    }
)
json.loads(rep.content.decode())

[['Q193175', ['斯捷潘·拉辛']],
 ['Q157280', ['斯捷潘·班傑拉', '斯捷潘·安德里約維奇·班傑拉', '班杰拉']],
 ['Q1064763', ['斯捷潘·尤克奇', 'Stjepan Jukić']],
 ['Q146496', ['斯捷潘·梅西奇', '斯傑潘·梅西奇']],
 ['Q347471', ['斯捷潘·拉迪奇']]]